In [1]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Dense



Using TensorFlow backend.


In [2]:
# open the json file
import json

with open('message.json') as f:
    data = json.load(f)



In [ ]:
def messageParse(message):
    message = message.replace("\u00c3\u00b0", "ð")
    message = message.replace("\u00c3\u00b0", "ð")
    
    message = message.replace("\u00c3\u00ad", "í")
    
    message = message.replace("\u00c3\u00b3", "ó")
    message = message.replace("\u00c3\u0093", "Ó")
    
    message = message.replace("\u00c3\u00a6", "æ")
    
    message = message.replace("\u00c3\u00a9", "é")
    
    message = message.replace("\u00c3\u00ba", "ú")
    
    message = message.replace("\u00c3\u00be", "þ")
    message = message.replace("\u00c3\u009e", "þ")
    
    message = message.replace("\u00c3\u00a1", "á")
    message = message.replace("\u00c2\u00b4", "á")
    
    message = message.replace("\u00c3\u00bd", "ý")
    message = message.replace("\u00c3\u009d", "ý")
    
    message = message.replace("\u00c3\u00b6", "ö")
    
    return message
    
        
        
    

In [1]:
def createMessageList(data, tmpDict):
    # Iterate all messages
    next = False
    index = len(tmpDict["timestamp_recieved"])
    
    for message in data["messages"]:
        if next:
            if "sender_name" in message:
                if not message["sender_name"] == "Vilhjalmur R. Vilhjalmsson":    
                    tmpDict["timestamp_recieved"][str(index)] = message["timestamp_ms"]
                    tmpDict["content_recieved"][str(index)] = message["content"]
                    index += 1
                
                next = False
        
        if "sender_name" in message:
            if (message['sender_name'] == "Vilhjalmur R. Vilhjalmsson"):
                if (('content' in message) and ('timestamp_ms' in message)):
                    next = True
                    tmpDict["timestamp_send"][str(index)] = message["timestamp_ms"]
                    tmpDict["content_sent"][str(index)] = message["content"]
            
  
    return tmpDict


In [4]:
# write to json
with open('data.json', 'w') as out:
    json.dump(outData, out)

NameError: name 'outData' is not defined

In [17]:
import os

fullList = { "timestamp_recieved": {}, "content_recieved": {}, "timestamp_send": {}, "content_sent": {} }
index = 0

root_path = os.path.abspath(os.path.curdir)
at_path = os.path.join(root_path, u'data/messages/archived_threads')
inbox_path = os.path.join(root_path, u'data/messages/inbox')

for dir in os.listdir(at_path):
    tmp_path = os.path.join(at_path, dir)
    if(os.path.exists(os.path.join(tmp_path, u'message.json'))):
        tmp_file = os.path.join(tmp_path, u'message.json')
        with open(str(tmp_file)) as f:
            data = json.load(f)
            fullList = createMessageList(data, fullList)

for dir in os.listdir(inbox_path):
    tmp_path = os.path.join(inbox_path, dir)
    if(os.path.exists(os.path.join(tmp_path, u'message.json'))):
        tmp_file = os.path.join(tmp_path, u'message.json')
        with open(str(tmp_file)) as f:
            data = json.load(f)
            fullList = createMessageList(data, fullList)


with open('data/data.json', 'w') as out:
    json.dump(fullList, out)


In [2]:
# The datasets


dataset = pd.read_json('data/data.json')
#with open('data.json', 'rU') as f:
#    pd.read_json(data)

print("Dataset length: " + str(len(dataset)))
dataset = dataset.drop(columns=["timestamp_send", "timestamp_recieved"], axis=1)


Dataset length: 6826


In [3]:
latent_dim = 256  # Latent dimensionality of the encoding space.
batch_size = 64
epochs = 1 # 100

input_texts = []
target_texts = []
input_words = set()
target_words = set()
i = 0

for index, row in dataset.iterrows():
    input_text = row["content_recieved"].split()
    input_texts.append(row["content_recieved"])
    for word in input_text:
        input_words.add(word)
    target_text = row["content_sent"].split()
    target_texts.append(row["content_sent"])
    for word in target_text:
        target_words.add(word)

word_list = list(input_words)
target_list = list(target_words)
num_encoder_tokens = len(word_list)
num_decoder_tokens = len(target_list)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(target_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
    

Number of samples: 6826
Number of unique input tokens: 14019
Number of unique output tokens: 11912
Max sequence length for inputs: 4119
Max sequence length for outputs: 1261


In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(word_list)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_list)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print(len(decoder_input_data[0][0]))

11912


In [8]:

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t, input_token_index[word]] = 1.
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[word]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [9]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')
print("=================")
print("DONE")

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Train on 5460 samples, validate on 1366 samples
Epoch 1/1


In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(x)
encoder_states = [state_h, state_c]



# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)



# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)



# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!


model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

# Save model
model.save('s2s.h5')